# Title

In [1]:
from env import Env
from RL_brain import DeepQNetwork

In [2]:
def run():
    step = 0
    for episode in range(100):
        # initial observation
        observation = env.reset()


        while True:
            # fresh env

            # RL choose action based on observation
            action = RL.choose_action(observation)
            if action == 0:
                action_ = "Channel_1"
            elif action == 1:
                action_ = "Channel_6"
            else:
                action_ = "Channel_11"
            print(action_)
            print(observation)
            # RL take action and get next observation and reward
            observation_, reward = env.step(action_)

            RL.store_transition(observation, action, reward, observation_)

            if (step > 200) and (step % 5 == 0):
                RL.learn()

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            step += 1


In [3]:
if __name__ == "__main__":
    # maze game
    env = Env()
    RL = DeepQNetwork(env.n_actions, env.n_features,
                      learning_rate=0.01,
                      reward_decay=0.9,
                      e_greedy=0.9,
                      replace_target_iter=200,
                      memory_size=2000,
                      # output_graph=True
                      )
    #print(env.n_actions)
    #print(env.n_features)
    run()
    RL.plot_cost()

Instructions for updating:
Colocations handled automatically by placer.
Channel_6
[-38.         -17.          24.          68.54327739]
Channel_6
[-19.         -14.          27.          89.30920008]
Channel_11
[-99.         -93.          20.          93.85759476]
Channel_11
[  0.         -79.          24.          14.70734771]
Channel_11
[-12.         -50.          27.          13.10623141]
Channel_6
[-89.          -9.          27.          47.37534233]
Channel_11
[-25.        -73.         24.          3.2917386]
Channel_11
[-21.         -77.          27.           6.73268165]
Channel_6
[-64.        -30.         27.         82.9945221]
Channel_6
[-22.          -3.          20.          49.10489899]
Channel_11
[ -2.        -26.         20.         51.3846588]
Channel_11
[-78.         -14.          24.          12.17198668]
Channel_11
[-66.         -73.          20.          35.61737472]
Channel_11
[-25.         -79.          24.          82.61120113]
Channel_6
[-46.         -14.       

[-14.         -12.          30.          82.74707059]
Channel_1
[-68.         -53.          24.          90.02616379]
Channel_11
[-94.         -61.          27.          45.16741395]
Channel_6
[-83.         -86.          27.          33.14769769]
Channel_11
[-80.         -73.          27.          74.52759804]
Channel_11
[-56.         -48.          24.          12.10705154]
Channel_1
[-12.         -64.          30.          65.28574961]
Channel_11
[-23.          -1.          27.           6.95577331]
Channel_6
[-13.        -27.         27.         58.4965158]
Channel_11
[-31.         -82.          24.           6.02822788]
Channel_11
[-88.         -59.          27.          42.36345905]
Channel_11
[-84.         -13.          24.          23.72853153]
Channel_11
[ -8.         -40.          30.          16.45738126]
Channel_11
[-92.         -30.          30.          23.50368032]
Channel_11
[-16.         -66.          20.          53.16273247]
Channel_11
[-73.        -75.         24.    

Channel_6
[-52.         -22.          24.          61.25858958]
Channel_11
[-17.         -23.          30.           7.33545116]
Channel_1
[-84.          -9.          27.          85.38476525]
Channel_6
[-17.         -21.          24.          91.48511707]
Channel_11
[-21.         -56.          20.           2.06124989]
Channel_11
[-65.         -14.          24.          28.60262437]
Channel_11
[-47.         -35.          27.          15.57036635]
Channel_6
[-91.         -61.          27.          93.04582604]
Channel_11
[-55.         -87.          20.          62.51089382]
Channel_11
[-49.         -99.          24.          11.82199414]
Channel_11
[-14.         -98.          30.          91.32153482]
Channel_6
[  -3.         -100.           24.           32.88303325]
Channel_11
[-67.         -30.          27.          55.92346331]
Channel_11
[-67.         -33.          30.           2.60937884]
Channel_1
[-83.         -45.          24.          52.70671572]
Channel_11
[-50.         -7

Channel_6
[-48.         -10.          20.          75.27434773]
Channel_11
[-49.         -39.          30.          91.27525466]
Channel_11
[-11.         -82.          27.          17.28732222]
Channel_11
[-44.         -90.          27.           3.53111575]
Channel_11
[-28.         -42.          24.          29.80937914]
Channel_11
[-28.         -50.          30.          93.30943056]
Channel_11
[ -7.         -25.          24.          17.26753181]
Channel_11
[-76.         -38.          30.          75.34584425]
Channel_11
[-80.         -93.          30.          39.14525841]
Channel_11
[-69.         -40.          24.          46.77516944]
Channel_6
[-94.         -82.          24.          95.68136564]
Channel_11
[-98.         -71.          30.          42.67279113]
Channel_11
[-20.         -62.          24.          52.86479396]
Channel_11
[-23.         -78.          20.          39.04506121]
Channel_11
[-49.         -67.          24.          32.70727051]
Channel_11
[-80.        -44


Channel_11
[-61.         -29.          20.          31.23854388]
Channel_11
[-51.         -52.          30.          27.51392549]
Channel_11
[-20.         -14.          24.          30.89749921]
Channel_11
[-18.         -95.          30.          99.43549357]
Channel_11
[-23.         -80.          24.          54.89838338]
Channel_11
[-46.        -68.         24.         98.0055285]
Channel_6
[-55.         -39.          20.          64.73831593]
Channel_11
[-53.           0.          27.          24.47653446]
Channel_11
[-20.         -71.          27.          83.87623141]
Channel_11
[-20.         -63.          30.          61.37126711]
Channel_11
[ -7.         -66.          27.          79.50058194]
Channel_11
[-26.         -74.          24.          30.91422388]
Channel_6
[-14.          -4.          30.          36.20821789]
Channel_6
[-55.        -21.         24.         69.7368608]
Channel_11
[ -4.         -93.          24.          50.49578457]
Channel_11
[-28.         -70.      

...

Channel_11
[ -5.         -15.          27.          33.48788776]
Channel_1
[-84.        -15.         27.         54.7395062]
Channel_11
[-37.         -96.          30.          29.27925305]
Channel_11
[ -1.         -12.          27.          84.93466394]
Channel_11
[-34.         -95.          27.          46.42668129]
Channel_11
[-17.         -42.          30.          68.31702559]
Channel_11
[-89.         -68.          24.           2.35488349]
Channel_11
[-34.         -30.          20.          26.80463769]
Channel_11
[ -42.         -100.           30.            7.93727889]
Channel_11
[-68.         -59.          24.          15.30902769]
Channel_1
[-67.         -32.          30.          13.94534627]
Channel_1
[-34.         -12.          30.          82.02702254]
Channel_1
[-89.         -29.          30.          85.51039133]
Channel_1
[-46.         -81.          20.          85.64474166]
Channel_1
[ -3.         -69.          27.          99.36867223]
Channel_11
[-37.        -27.   

Channel_1
[-13.         -13.          20.          28.83027867]
Channel_1
[-48.         -54.          27.          40.45049248]
Channel_11
[-69.          -1.          20.          84.68880772]
Channel_1
[-31.         -45.          27.           7.79368359]
Channel_6
[-98.         -14.          24.          52.54050291]
Channel_1
[-32.          -6.          30.          42.51008017]
Channel_1
[-16.         -39.          30.          21.72219397]
Channel_1
[-99.          -6.          20.          88.06577939]
Channel_11
[-34.         -42.          24.          90.42180892]
Channel_11
[-100.          -47.           27.           33.43317398]
Channel_11
[-79.         -98.          24.          15.58229105]
Channel_1
[-21.         -72.          20.          21.80004981]
Channel_11
[-69.          -5.          27.          37.80551713]
Channel_1
[  0.         -39.          27.          20.25429383]
Channel_1
[-32.         -86.          20.          14.60858422]
Channel_6
[-82.         -62.   

Channel_11
[-93.       -60.        27.        88.190007]
Channel_1
[-43.         -97.          24.          69.50972585]
Channel_1
[-57.         -62.          24.          35.16674621]
Channel_1
[-19.         -19.          20.          55.92992946]
Channel_1
[-52.        -25.         27.         88.1174983]
Channel_1
[-83.         -63.          27.          18.63635538]
Channel_11
[-59.          -5.          30.          75.09776602]
Channel_1
[-57.         -73.          27.          63.79713441]
Channel_1
[-44.         -81.          24.          47.21106749]
Channel_1
[-18.         -63.          30.          37.15418388]
Channel_11
[-90.         -38.          27.          40.24352816]
Channel_11
[-29.          -3.          24.           7.04766439]
Channel_11
[-18.          -5.          27.          20.05569951]
Channel_11
[-10.         -33.          27.          15.45316703]
Channel_11
[-88.         -82.          30.          91.06030983]
Channel_6
[-94.         -58.          24.    

Channel_1
[-91.         -33.          24.          10.84343105]
Channel_1
[-83.         -18.          24.          91.48591403]
Channel_1
[-87.         -89.          30.          83.93489999]
Channel_1
[-85.         -69.          30.          36.52398229]
Channel_6
[-38.         -27.          30.          67.62061936]
Channel_1
[-100.          -14.           27.           38.13216742]
Channel_1
[ -5.         -94.          24.          67.20387412]
Channel_6
[-88.         -71.          27.          94.03556043]
Channel_1
[-71.          -8.          27.          34.47095313]
Channel_1
[-21.         -45.          24.          66.62432172]
Channel_1
[-83.         -82.          20.          18.65741588]
Channel_11
[ -2.         -24.          27.          56.79746651]
Channel_6
[-91.        -83.         24.         64.3340976]
Channel_1
[-81.         -31.          27.          65.74829526]
Channel_11
[-53.          -8.          30.          44.28519382]
Channel_1
[-52.         -46.          

KeyboardInterrupt: 